https://stepik.org/lesson/1576183/step/9

In [ ]:
# @title загрузка датасета Digits в переменную
from sklearn.datasets import load_digits
import torch


digits = load_digits()

# Признаки и целевые значения
_global_var_data_x = torch.tensor(digits.data, dtype=torch.float32)
_global_var_target = torch.tensor(digits.target, dtype=torch.int64)

# Размеры и типы
print("Размеры и типы:", _global_var_data_x.shape, _global_var_data_x.dtype, _global_var_target.shape, _global_var_target.dtype)

# Прочие характеристики
print("Количество классов: ", len(digits.target_names))
print("Названия классов: ", digits.target_names)
print("Названия признаков: ", digits.feature_names)

# # Первые строки данных
# print("Первые 5 строк признаков:\n", _global_var_data_x[:5])
# print("Первые 5 целевых значений:\n", _global_var_target[:5])

In [ ]:
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim


class ThreeLayerModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(64, 64)
        self.layer2 = nn.Linear(64, 32)
        self.out = nn.Linear(32, 10)
        self.dropout_1 = nn.Dropout1d(0.3)
        self.dropout_2 = nn.Dropout1d(0.3)

    def forward(self, x):
        x = self.layer1(x).relu()
        x = self.dropout_1(x)
        x = self.layer2(x).relu()
        x = self.dropout_2(x)
        x = self.out(x)
        return x

# Создание объекта класса data.TensorDataset с разбиением на две части
ds = data.TensorDataset(_global_var_data_x, _global_var_target)
d_train, d_test = data.random_split(ds, [0.7, 0.3])

# Создание объектов класса DataLoader
train_data = data.DataLoader(d_train, batch_size=16, shuffle=True, drop_last=False)
test_data = data.DataLoader(d_test, batch_size=len(d_test), shuffle=False)

# Модель
model = ThreeLayerModel()
model.train()

# Оптимизатор Adam и функция потерь CrossEntropyLoss
optimizer = optim.Adam(params=model.parameters(), weight_decay=0.1, lr=0.01)
loss_func = nn.CrossEntropyLoss()

# Обучение модели (градиентный спуск)
for _e in range(2): # epochs=2
    for _x, _y in train_data:
        pred = model(_x).squeeze()  # прогноз по батчу
        loss = loss_func(pred, _y)  # потери по батчу
        optimizer.zero_grad()       # очистка градиента
        loss.backward()
        optimizer.step()

# Доля верных классификаций обученной модели по тестовой выборке
model.eval()
loss_test = 0 # для подсчёта числа верных прогнозов
for _x, _y in test_data:
    pred = model(_x)
    loss_test += (pred.argmax(dim=1) == _y).sum().item()

# Качество
Q = loss_test / len(d_test)
# print(f"Качество: {Q}")

Качество: 0.7810760667903525
